In [83]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_columns', None)

file_path = 'data/raw/pede_2022-24.xlsx'
df_2022 = pd.read_excel(file_path, sheet_name='PEDE2022')
df_2023 = pd.read_excel(file_path, sheet_name='PEDE2023')
df_2024 = pd.read_excel(file_path, sheet_name='PEDE2024')

In [84]:
def clean_fase(valor):
    if pd.isna(valor):
        return np.nan
    
    str_val = str(valor).upper().strip()
    
    if 'ALFA' in str_val:
        return 0
    
    match = re.search(r'(\d+)', str_val)
    
    if match:
        return int(match.group(1))
    
    return np.nan

def clean_idade(valor):
    if pd.isna(valor):
        return valor
    
    str_val = str(valor)
    
    if str_val.startswith('1900-01-'):
        try:
            return pd.to_datetime(str_val).day
        except (ValueError, TypeError):
            return np.nan
            
    try:
        return int(float(str_val))
    except (ValueError, TypeError):
        return valor
    
def clean_genero(valor):
    map_generos = {
        'Menina': 'F', 
        'Menino': 'M', 
        'Feminino': 'F', 
        'Masculino': 'M',
    }
    
    if pd.isna(valor):
        return np.nan
    
    return map_generos.get(valor)

def clean_RA(valor):
    if pd.isna(valor):
        return np.nan
    
    str_val = str(valor).upper().strip()
    
    match = re.search(r'(\d+)', str_val)
    
    if match:
        return int(match.group(1))
    
    return np.nan


def clean_pedra(valor):
    if pd.isna(valor):
        return None
    
    if valor == 'Agata':
        return 'Ágata'
    
    if valor == 'INCLUIR':
        return None

    else:
        return valor
    
    
def clean_INDE(valor):
    if pd.isna(valor):
        return np.nan
    if valor == 'INCLUIR':
        return np.nan
    else:
        return float(valor)
    
    
def corrigir_instituicao(valor):
    map_instituicao = {
        'Escola Pública': 'Pública', 
        'Pública': 'Pública',
        'Rede Decisão': 'Privada', 
        'Escola JP II': 'Privada', 
        'Privada': 'Privada', 
        'Privada - Programa de Apadrinhamento': 'Bolsista',
        'Privada - Programa de apadrinhamento': 'Bolsista', 
        'Privada *Parcerias com Bolsa 100%': 'Bolsista',
        'Privada - Pagamento por *Empresa Parceira': 'Bolsista', 
        'Bolsista Universitário *Formado (a)': 'Bolsista',
        'Concluiu o 3º EM': 'Outros',
        'Nenhuma das opções acima': 'Outros', 
    }
    if pd.isna(valor):
        return 'Outros'
    
    if valor in map_instituicao:
        return map_instituicao[valor]
    
    return 'Outros'


In [89]:
# Map columns to a common format
RA = 'RA'
ANO_DADOS = 'ano_dados'
FASE = 'fase'
IDADE = 'idade'
GENERO = 'genero'
ANO_INGRESSO = 'ano_ingresso'
INSTITUICAO = 'instituicao'
PEDRA_ATUAL = 'pedra_atual'
INDE = 'inde_atual'
IAA = 'indicador_auto_avaliacao'
IEG = 'indicador_engajamento'
IPS = 'indicador_psicossocial'
IDA = 'indicador_aprendizagem'
IPV = 'indicador_ponto_virada'
IAN = 'indicador_adequacao_nivel'
IPP ='indicador_psico_pedagogico'
DEFASAGEM = 'defasagem'

columns_to_keep = [
    RA,
    ANO_DADOS,
    FASE,
    IDADE,
    GENERO,
    ANO_INGRESSO,
    INSTITUICAO,
    PEDRA_ATUAL,
    INDE,
    IAA,
    IEG,
    IPS,
    IDA,
    IPV,
    IAN,
    IPP,
    DEFASAGEM,
]

# 2022
map_columns_2022 = {
    'RA': RA,
    'Fase': FASE,
    'Idade 22': IDADE,
    'Gênero': GENERO,
    'Ano ingresso': ANO_INGRESSO,
    'Instituição de ensino': INSTITUICAO,
    'Pedra 22': PEDRA_ATUAL,
    'INDE 22': INDE,
    'IAA': IAA,
    'IEG': IEG,
    'IPS': IPS,
    'IDA': IDA,
    'IPV': IPV,
    'IAN': IAN,
    'IPP': IPP,
    'Defas': DEFASAGEM,
}

df_2022_clean = df_2022.rename(columns=map_columns_2022)
df_2022_clean[ANO_DADOS] = 2022
df_2022_clean[IPP] = (
    (
        df_2022_clean[INDE] - (
            df_2022_clean[IAN] * 0.1 + 
            df_2022_clean[IDA] * 0.2 + 
            df_2022_clean[IEG] * 0.2 + 
            df_2022_clean[IAA] * 0.1 + 
            df_2022_clean[IPS] * 0.1 + 
            df_2022_clean[IPV] * 0.2
        )
    ) / 0.1
).round(3)
df_2022_clean = df_2022_clean[columns_to_keep]

# 2023
map_columns_2023 = {
    'RA': RA,
    'Fase': FASE,
    'Idade': IDADE,
    'Gênero': GENERO,
    'Ano ingresso': ANO_INGRESSO,
    'Instituição de ensino': INSTITUICAO,
    'Pedra 2023': PEDRA_ATUAL,
    'INDE 2023': INDE,
    'IAA': IAA,
    'IEG': IEG,
    'IPS': IPS,
    'IDA': IDA,
    'IPV': IPV,
    'IAN': IAN,
    'IPP': IPP,
    'Defasagem': DEFASAGEM,
}

df_2023_clean = df_2023.rename(columns=map_columns_2023)
df_2023_clean[ANO_DADOS] = 2023
df_2023_clean = df_2023_clean[columns_to_keep]

# 2024
map_columns_2024 = {
    'RA': RA,
    'Fase': FASE,
    'Idade': IDADE,
    'Gênero': GENERO,
    'Ano ingresso': ANO_INGRESSO,
    'Instituição de ensino': INSTITUICAO,
    'Pedra 2024': PEDRA_ATUAL,
    'INDE 2024': INDE,
    'IAA': IAA,
    'IEG': IEG,
    'IPS': IPS,
    'IDA': IDA,
    'IPV': IPV,
    'IAN': IAN,
    'IPP': IPP,
    'Defasagem': DEFASAGEM,
}

df_2024_clean = df_2024.rename(columns=map_columns_2024)
df_2024_clean[ANO_DADOS] = 2024
df_2024_clean = df_2024_clean[columns_to_keep]

df = pd.concat([df_2022_clean, df_2023_clean, df_2024_clean], ignore_index=True)

df[IDADE] = df[IDADE].apply(clean_idade)
df[GENERO] = df[GENERO].apply(clean_genero)
df[FASE] = df[FASE].apply(clean_fase)
df[RA] = df[RA].apply(clean_RA)
df[PEDRA_ATUAL] = df[PEDRA_ATUAL].apply(clean_pedra)
df[ANO_INGRESSO] = df[ANO_DADOS] - df[ANO_INGRESSO]
df[INDE] = df[INDE].apply(clean_INDE)
df[INSTITUICAO] = df[INSTITUICAO].apply(corrigir_instituicao)
df.rename(columns={ANO_INGRESSO: 'anos_ingresso'}, inplace=True)
df.to_csv('data/processed/pede_2022-24.csv', index=False, sep=';', decimal=',')

In [93]:
df.sort_values([RA, ANO_DADOS])#.loc[df[ANO_DADOS] == 2023]

,RA,ano_dados,fase,idade,genero,anos_ingresso,instituicao,pedra_atual,inde_atual,indicador_auto_avaliacao,indicador_engajamento,indicador_psicossocial,indicador_aprendizagem,indicador_ponto_virada,indicador_adequacao_nivel,indicador_psico_pedagogico,defasagem
0,1,2022,7,19,F,6,Pública,Quartzo,5.783,8.3,4.1,5.6,4.0,7.278,5.0,8.174,-1
1855,1,2023,8,20,F,7,Bolsista,None,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,0
2958,1,2024,8,21,F,3,Bolsista,None,NaN,NaN,0.0,NaN,NaN,NaN,10.0,NaN,0
1,2,2022,7,17,F,5,Privada,Ametista,7.055,8.8,5.2,6.3,6.8,6.778,10.0,7.894,0
1869,2,2023,8,18,F,6,Bolsista,None,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3024,1657,2024,9,20,M,3,Bolsista,None,NaN,NaN,0.0,NaN,NaN,NaN,10.0,NaN,1
3025,1658,2024,9,21,M,3,Privada,None,NaN,NaN,0.0,NaN,NaN,NaN,10.0,NaN,1
3026,1659,2024,9,21,M,3,Bolsista,None,NaN,NaN,0.0,NaN,NaN,NaN,10.0,NaN,1
3028,1660,2024,9,24,F,3,Bolsista,None,NaN,NaN,0.0,NaN,NaN,NaN,10.0,NaN,1


In [78]:
df[INSTITUICAO].value_counts()

instituicao
Pública     1670
Privada      184
Bolsista     159
Outros         8
Name: count, dtype: int64